In [36]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
import os # Инициализация библиотек

vid_path = 1 # № камеры
cam = cv2.VideoCapture(vid_path) #Инициализация камеры

success, backgr = cam.read() #Запуск камеры
kernel_er = np.ones((1, 1), np.uint8) 
kernel_dil = np.ones((1, 1), np.uint8)

cv2.namedWindow("clean") # Создание окна
cv2.createTrackbar("max_objects", "clean", 10, 25, lambda i: i) # Создание ползунков
cv2.createTrackbar("min_size", "clean", 40, 2000, lambda i: i)
cv2.createTrackbar("th", "clean", 30, 255, lambda i: i) 

cv2.createTrackbar("lh", "clean", 10, 255, lambda i: i) 
cv2.createTrackbar("hh", "clean", 135, 255, lambda i: i)

cv2.createTrackbar("ls", "clean", 20, 255, lambda i: i) 
cv2.createTrackbar("hs", "clean", 255, 255, lambda i: i)

cv2.createTrackbar("lv", "clean", 230, 255, lambda i: i) 
cv2.createTrackbar("hv", "clean", 255, 255, lambda i: i) 

filename = 1
path = r'D:\Frames' # Путь сохранения кадров
os.chdir(path) 

while(True):
    success, frame = cam.read() # Запуск камеры
   
    blurred = cv2.blur(frame, (11, 11)) # Добавление размытия
    
    if (success == False): 
        break # Если камера не открылась - завершить

    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV) # Перевод в HSV
    
    max_objects = cv2.getTrackbarPos("max_objects", "clean") # Получение значений с ползунков
    min_size = cv2.getTrackbarPos("min_size", "clean")
    th = cv2.getTrackbarPos("th", "clean")
    lh = cv2.getTrackbarPos("lh", "clean")
    ls = cv2.getTrackbarPos("ls", "clean")
    lv = cv2.getTrackbarPos("lv", "clean")
    hh = cv2.getTrackbarPos("hh", "clean")
    hs = cv2.getTrackbarPos("hs", "clean")
    hv = cv2.getTrackbarPos("hv", "clean")
    
    backgr = cv2.addWeighted(backgr, 0.97, blurred, 0.03, 0) # Вычитание фона
    diff = cv2.absdiff(backgr, blurred) 
    mask = cv2.inRange(backgr, (th, th, th), (255, 255, 255)) 
    maskV = cv2.inRange(blurred,  (lh ,ls, lv),( hh, hs, hs)) # Создание маски по цвету
    maskC = cv2.bitwise_and(mask, maskV) # Объединение масок

    dilated = cv2.morphologyEx(maskC, cv2.MORPH_DILATE, kernel_dil) # Наращивание
    erosed = cv2.morphologyEx(maskC, cv2.MORPH_ERODE, kernel_er) # Эрозия
    
    contours, _ = cv2.findContours(erosed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Поиск контуров

    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_size] # Фильтрация контуров
    filtered_contours = sorted(filtered_contours, key=cv2.contourArea, reverse=True)[:max_objects] 

    output = cv2.connectedComponentsWithStats(erosed) # Вывод
    
    for cnt in filtered_contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2) # Создание прямоугольника на экране
        
    filename = filename + 1
    cv2.imwrite("Frame_mask"+str(filename)+".jpg", maskC) # сохранение кадров
    cv2.imwrite("Frame_clear"+str(filename)+".jpg", frame) 

    # Вывод изображений
    
    #cv2.imshow("backgr", backgr) 
    #cv2.imshow("output", cv2.absdiff(backgr,frame))
    #cv2.imshow("mask", mask)
    cv2.imshow("clean", frame) 
    #cv2.imshow("maskV", maskV)
    cv2.imshow("maskC", maskC)
    
    key = cv2.waitKey(30) & 0xFF
    
    if (key == ord('q')):
        break # Закрытие программы

# Освобождение камеры

cam.release() 
cv2.destroyAllWindows()
cv2.waitKey(10)

-1